In [4]:
from blazingsql import BlazingContext
from dask_cuda import LocalCUDACluster
from dask.distributed import Client

cluster = LocalCUDACluster(CUDA_VISIBLE_DEVICES=range(0,8))
client = Client(cluster)
bc = BlazingContext(dask_client=client, network_interface='ib0')

/home/christian/miniconda3/envs/bsql/lib/python3.8/site-packages/distributed/node.py:151: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 45741 instead
  warnings.warn(


BlazingContext ready


In [3]:
data_dir = "/raid/tpcx-bb/sf1000/parquet_1gb/" # 1K

## Q12

In [6]:
q12_i_category_IN = "'Books', 'Electronics'"

In [7]:
bc.create_table("web_clickstreams", data_dir + "/web_clickstreams/*.parquet")
bc.create_table("store_sales", data_dir + "/store_sales/*.parquet")
bc.create_table("item", data_dir + "/item/*.parquet")

In [8]:
query = f"""
    SELECT DISTINCT wcs_user_sk
    FROM
    (
        SELECT DISTINCT
            wcs_user_sk,
            wcs_click_date_sk
        FROM web_clickstreams, item
        WHERE wcs_click_date_sk BETWEEN 37134 AND 37164
        AND i_category IN ({q12_i_category_IN})
        AND wcs_item_sk = i_item_sk
        AND wcs_user_sk IS NOT NULL
        AND wcs_sales_sk IS NULL
    ) webInRange,
    (
        SELECT DISTINCT
            ss_customer_sk,
            ss_sold_date_sk
        FROM store_sales, item
        WHERE ss_sold_date_sk BETWEEN 37134 AND 37224
        AND i_category IN ({q12_i_category_IN})
        AND ss_item_sk = i_item_sk
        AND ss_customer_sk IS NOT NULL
    ) storeInRange
    WHERE wcs_user_sk = ss_customer_sk
    AND wcs_click_date_sk < ss_sold_date_sk
    ORDER BY wcs_user_sk
"""
result = bc.sql(query)

In [9]:
result.head()

,wcs_user_sk
0,2
1,164
2,394
3,623
4,650


## Q21

In [10]:
bc.create_table("store_sales", data_dir + "/store_sales/*.parquet")
bc.create_table("date_dim", data_dir + "/date_dim/*.parquet")
bc.create_table("item", data_dir + "/item/*.parquet")
bc.create_table("web_sales", data_dir + "/web_sales/*.parquet")
bc.create_table("store_returns", data_dir + "/store_returns/*.parquet")
bc.create_table("store", data_dir + "/store/*.parquet")

In [11]:
query = """
    SELECT
        part_i.i_item_id AS i_item_id,
        part_i.i_item_desc AS i_item_desc,
        part_s.s_store_id AS s_store_id,
        part_s.s_store_name AS s_store_name,
        CAST(SUM(part_ss.ss_quantity) AS BIGINT) AS store_sales_quantity,
        CAST(SUM(part_sr.sr_return_quantity) AS BIGINT) AS store_returns_quantity,
        CAST(SUM(part_ws.ws_quantity) AS BIGINT) AS web_sales_quantity
    FROM 
    (
        SELECT
            sr_item_sk,
            sr_customer_sk,
            sr_ticket_number,
            sr_return_quantity
        FROM
            store_returns sr,
            date_dim d2
        WHERE d2.d_year = 2003
        AND d2.d_moy BETWEEN 1 AND 7 --which were returned in the next six months
        AND sr.sr_returned_date_sk = d2.d_date_sk
    ) part_sr
    INNER JOIN 
    (
        SELECT
            ws_item_sk,
            ws_bill_customer_sk,
            ws_quantity
        FROM
            web_sales ws,
            date_dim d3
        -- in the following three years (re-purchased by the returning customer afterwards through the web sales channel)
        WHERE d3.d_year BETWEEN 2003 AND 2005 
        AND ws.ws_sold_date_sk = d3.d_date_sk
    ) part_ws ON 
    (
        part_sr.sr_item_sk = part_ws.ws_item_sk
        AND part_sr.sr_customer_sk = part_ws.ws_bill_customer_sk
    ) INNER JOIN 
    (
        SELECT
            ss_item_sk,
            ss_store_sk,
            ss_customer_sk,
            ss_ticket_number,
            ss_quantity
        FROM
            store_sales ss,
            date_dim d1
        WHERE d1.d_year = 2003
        AND d1.d_moy = 1
        AND ss.ss_sold_date_sk = d1.d_date_sk
    ) part_ss ON 
    (
        part_ss.ss_ticket_number = part_sr.sr_ticket_number
        AND part_ss.ss_item_sk = part_sr.sr_item_sk
        AND part_ss.ss_customer_sk = part_sr.sr_customer_sk
    )
    INNER JOIN store part_s ON 
    (
        part_s.s_store_sk = part_ss.ss_store_sk
    )
    INNER JOIN item part_i ON 
    (
        part_i.i_item_sk = part_ss.ss_item_sk
    )
    GROUP BY
        part_i.i_item_id,
        part_i.i_item_desc,
        part_s.s_store_id,
        part_s.s_store_name
    ORDER BY
        part_i.i_item_id,
        part_i.i_item_desc,
        part_s.s_store_id,
        part_s.s_store_name
    LIMIT 100
"""
result = bc.sql(query)

In [12]:
result.head()

,i_item_id,i_item_desc,s_store_id,s_store_name,store_sales_quantity,store_returns_quantity,web_sales_quantity
0,AAAAAAAAAAAAABBT,tithes can thrash ruthlessly permanent warthog...,AAAAAAAAAAAAAAAI,Williamstown,95,95,41
1,AAAAAAAAAAAAAFIE,sly quick warthogs should engage idle quick pl...,AAAAAAAAAAAAAAAF,Egypt,22,21,17
2,AAAAAAAAAAAAAGWB,idle stealthy pinto beans beside the slowly th...,AAAAAAAAAAAAAAAK,Avalon,26,12,45
3,AAAAAAAAAAAAAJXF,idle Tiresias' detect? enticing excuses use sl...,AAAAAAAAAAAAAAAB,Hillside,68,47,77
4,AAAAAAAAAAAAAOUQ,stealthily quick foxes shall thrash blithe bli...,AAAAAAAAAAAAAAAG,Bloomfield,306,198,377
